In [1]:
from poor_man_lakehouse.config import settings
from poor_man_lakehouse.spark_connector.builder import retrieve_current_spark_session

spark = retrieve_current_spark_session()

:: loading settings :: url = jar:file:/home/montanaro/workspace/poor-man-lakehouse/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/montanaro/.ivy2.5.2/cache
The jars for the packages stored in: /home/montanaro/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-4.0_2.13 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.postgresql#postgresql added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.13 added as a dependency
io.unitycatalog#unitycatalog-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b2cd6be8-e006-401e-ab3a-2979fa5e0335;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.1 in central
	found io.delta#delta-storage;4.0.1 in central
	found org.antlr#antlr4-run

In [ ]:
test_df = spark.createDataFrame(
    [
        (1, "Alice"),
        (2, "Bob"),
        (3, "Charlie"),
    ],
    ["id", "name"],
)
test_df.show()

26/02/14 21:51:44 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:51:59 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:52:14 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [2]:
spark.sql("SHOW CATALOGS").show()
spark.sql(f"show schemas in {settings.CATALOG}").show()
spark.sql("show schemas in spark_catalog").show()
spark.sql(f"SHOW  tables in {settings.CATALOG}.default").show()

+-------------+
|      catalog|
+-------------+
|   lakekeeper|
|spark_catalog|
+-------------+

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [3]:
from datetime import UTC, datetime

import polars as pl
from pyspark.sql import DataFrame

data = pl.DataFrame(
    {
        "datetime": [
            datetime(2023, 1, 1, 12, 0, tzinfo=UTC),
            datetime(2023, 1, 2, 12, 0, tzinfo=UTC),
            datetime(2023, 1, 3, 12, 0, tzinfo=UTC),
        ],
        "symbol": ["AAPL", "GOOGL", "MSFT"],
        "bid": [150.0, 2800.0, 300.0],
        "ask": [151.0, 2805.0, 305.0],
        "details": [
            {"created_by": "user1"},
            {"created_by": "user2"},
            {"created_by": None},
        ],
    },
)

spark_df: DataFrame = spark.createDataFrame(data.to_pandas())


In [4]:
spark_df.write.format("iceberg").mode("overwrite").saveAsTable(
    f"{settings.CATALOG}.default.prova", mode="overwrite"
)

26/02/14 21:38:14 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/montanaro/workspace/poor-man-lakehouse/.venv/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/montanaro/workspace/poor-man-lakehouse/.venv/lib/python3.13/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/home/montanaro/.local/share/uv/python/cpython-3.13.11-linux-x86_64-gnu/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt
26/02/14 21:38:20 ERROR OverwriteByExpressionExec: Data source write support IcebergBatchWrit

KeyboardInterrupt: 

26/02/14 21:38:29 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:38:44 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:38:59 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:39:14 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:39:29 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/02/14 21:39:44 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [5]:
spark.sql(f"""SELECT * FROM {settings.CATALOG}.default.prova""").show()

+-------------------+------+------+------+--------------------+
|           datetime|symbol|   bid|   ask|             details|
+-------------------+------+------+------+--------------------+
|2023-01-01 12:00:00|  AAPL| 150.0| 151.0|{created_by -> us...|
|2023-01-02 12:00:00| GOOGL|2800.0|2805.0|{created_by -> us...|
|2023-01-03 12:00:00|  MSFT| 300.0| 305.0|{created_by -> NULL}|
+-------------------+------+------+------+--------------------+



## Testing LakeSail

To test Lakesail, it's necessary to restart the kernel since the sparkSession was already created and we need a clean one.

In [ ]:
from pysail.spark import SparkConnectServer
from pyspark.sql import SparkSession

server = SparkConnectServer()
server.start()
address = server.listening_address
if address is None:
    raise RuntimeError("Failed to start Spark Connect server")
_, port = address

spark = SparkSession.builder.remote(f"sc://localhost:{port}").getOrCreate()


In [ ]:
path = "s3a://warehouse/test_parquet"
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], schema="id INT, name STRING")
df.write.parquet(path)

df = spark.read.parquet(path)
